# Play with plotting latent dimensions

In [30]:
import os
import sys
os.chdir("/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/bin")
sys.path.append(os.getcwd()) 
import torch

from flower.task import Net, train
import torch
from lib.plot import Visualizations
from torch.utils.data import DataLoader
from lib.collate_functions import basic_collate_fn
from lib.physionet import variable_time_collate_fn
import lib.utils as utils

v = Visualizations(device = "cpu")

model = Net()

#################################################################
# Load data
################################################################
#################################################################
path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
results_path = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/results_freq_noise"
prefix = "freq_amp_decay_maxfreq_0.1"

train_data_0 = torch.load(f"{path_prefix}/{prefix}/client_0_train.pt", weights_only= True)
train_data_1 = torch.load(f"{path_prefix}/{prefix}/client_1_train.pt", weights_only= True)
timesteps = torch.load(f"{path_prefix}/{prefix}/client_0_time_steps_train.pt", weights_only= True)
timesteps = timesteps[0]  # Assuming both clients have the same timesteps
dataset_name = "test"
sample_tp = 0.5
cut_tp = None
extrap = None
batch_size = 50

data_loader = DataLoader(train_data_0, batch_size = batch_size, shuffle=False,
    collate_fn= lambda batch: basic_collate_fn(batch, timesteps, dataset_name, sample_tp, cut_tp, extrap, data_type = "test"))
data_loader = utils.inf_generator(data_loader)
test_dict = utils.get_next_batch(data_loader)

Model configuration: {'dataset_name': 'physionet_filtered', 'data_folder': '../data/physionet_filtered/client_0', 'sample_tp': '0.5', 'lr': '0.01', 'lrdecay': '0.99', 'cut_tp': None, 'batch_size': '50', 'extrap': False, 'obsrv_std': '0.01', 'poisson': False, 'rec_layers': '1', 'gen_layers': '1', 'units': '100', 'gru_units': '100', 'latents': '10', 'rec_dims': '20', 'z0_encoder': 'odernn', 'train_classif_w_reconstr': False, 'classif': False, 'linear_classif': False, 'classif_per_tp': False, 'n_labels': '1', 'input_dim': '1', 'aggregation': 'FedAvg', 'storeweights': False, 'alpha': '0.5', 'localepochs': '3', 'serverrounds': '3', 'fractionfit': '1.0', 'fractionevaluate': '1.0', 'device': 'cpu', 'use_wandb': False, 'gradientclipping': False, 'decay_onset': '100'}
GRU unit: latent_dim 20, input_dim 2, n_units 100


In [ ]:


	def draw_all_plots_one_dim(self, data_dict, model,
		plot_name = "", save = False, experimentID = 0.):

		data =  data_dict["data_to_predict"]
		time_steps = data_dict["tp_to_predict"]
		mask = data_dict["mask_predicted_data"]
		# print the mask
		
		observed_data =  data_dict["observed_data"]
		observed_time_steps = data_dict["observed_tp"]
		observed_mask = data_dict["observed_mask"]
		# print the mask 

		device = get_device(time_steps)

		time_steps_to_predict = time_steps
		if isinstance(model, LatentODE):
			# sample at the original time points
			time_steps_to_predict = utils.linspace_vector(time_steps[0], time_steps[-1], 100).to(device)

		reconstructions, info = model.get_reconstruction(time_steps_to_predict, 
			observed_data, observed_time_steps, mask = observed_mask, n_traj_samples = 10)

		n_traj_to_show = 3
		# plot only 10 trajectories
		data_for_plotting = observed_data[:n_traj_to_show]
		mask_for_plotting = observed_mask[:n_traj_to_show]

		data_full = data[:n_traj_to_show]

		reconstructions_for_plotting = reconstructions.mean(dim=0)[:n_traj_to_show]
		reconstr_std = reconstructions.std(dim=0)[:n_traj_to_show]

		dim_to_show = 0
		max_y = max(
			data_for_plotting[:,:,dim_to_show].cpu().numpy().max(),
			reconstructions[:,:,dim_to_show].cpu().numpy().max())
		min_y = min(
			data_for_plotting[:,:,dim_to_show].cpu().numpy().min(),
			reconstructions[:,:,dim_to_show].cpu().numpy().min())

		############################################
		# Plot reconstructions, true postrior and approximate posterior

		cmap = plt.colormaps['Set1']
		for traj_id in range(3):

			# Plot observations
			plot_trajectories(self.ax_traj[traj_id], 
				data_full[traj_id].unsqueeze(0), observed_time_steps, 
				#mask = mask_for_plotting[traj_id].unsqueeze(0),
				min_y = min_y, max_y = max_y, #title="True trajectories", 
				marker = 'o', linestyle='', dim_to_show = dim_to_show, markersize= 5,
				color = cmap(4))
			# # # Plot observations
			plot_trajectories(self.ax_traj[traj_id], 
				data_for_plotting[traj_id].unsqueeze(0), observed_time_steps, 
				mask = mask_for_plotting[traj_id].unsqueeze(0),
				min_y = min_y, max_y = max_y, #title="True trajectories", 
				marker = 'o', linestyle='', dim_to_show = dim_to_show, add_to_plot= True, markersize= 2,
				color = cmap(5))

			# # Plot reconstructions
			plot_trajectories(self.ax_traj[traj_id],
				reconstructions_for_plotting[traj_id].unsqueeze(0), time_steps_to_predict, 
				min_y = min_y, max_y = max_y, title="Sample {} (data space)".format(traj_id), dim_to_show = dim_to_show,
				add_to_plot = True, marker = '', color = cmap(3), linewidth = 1)
			# # Plot variance estimated over multiple samples from approx posterior
			plot_std(self.ax_traj[traj_id], 
				reconstructions_for_plotting[traj_id].unsqueeze(0), reconstr_std[traj_id].unsqueeze(0), 
				time_steps_to_predict, alpha=0.5, color = cmap(3), add_to_plot= True)
			self.set_plot_lims(self.ax_traj[traj_id], "traj_" + str(traj_id))
			
			# Plot true posterior and approximate posterior
			# self.draw_one_density_plot(self.ax_density[traj_id],
			# 	model, data_dict, traj_id = traj_id,
			# 	multiply_by_poisson = False)
			# self.set_plot_lims(self.ax_density[traj_id], "density_" + str(traj_id))
			# self.ax_density[traj_id].set_title("Sample {}: p(z0) and q(z0 | x)".format(traj_id))
		############################################
		# Get several samples for the same trajectory
		# one_traj = data_for_plotting[:1]
		# first_point = one_traj[:,0]

		# samples_same_traj, _ = model.get_reconstruction(time_steps_to_predict, 
		# 	observed_data[:1], observed_time_steps, mask = observed_mask[:1], n_traj_samples = 5)
		# samples_same_traj = samples_same_traj.squeeze(1)
		
		# plot_trajectories(self.ax_samples_same_traj, samples_same_traj, time_steps_to_predict, marker = '')
		# plot_trajectories(self.ax_samples_same_traj, one_traj, time_steps, linestyle = "", 
		# 	label = "True traj", add_to_plot = True, title="Reconstructions for the same trajectory (data space)")

		############################################
		# Plot trajectories from prior
		
		if isinstance(model, LatentODE):
			torch.manual_seed(1991)
			np.random.seed(1991)

			traj_from_prior = model.sample_traj_from_prior(time_steps_to_predict, n_traj_samples = 3)
			# Since in this case n_traj = 1, n_traj_samples -- requested number of samples from the prior, squeeze n_traj dimension
			traj_from_prior = traj_from_prior.squeeze(1)

			plot_trajectories(self.ax_traj_from_prior, traj_from_prior, time_steps_to_predict, 
				marker = '', linewidth = 3)
			self.ax_traj_from_prior.set_title("Samples from prior (data space)", pad = 20)
			#self.set_plot_lims(self.ax_traj_from_prior, "traj_from_prior")
		################################################

		# Plot z0
		# first_point_mu, first_point_std, first_point_enc = info["first_point"]

		# dim1 = 0
		# dim2 = 1
		# self.ax_z0.cla()
		# # first_point_enc shape: [1, n_traj, n_dims]
		# self.ax_z0.scatter(first_point_enc.cpu()[0,:,dim1], first_point_enc.cpu()[0,:,dim2])
		# self.ax_z0.set_title("Encodings z0 of all test trajectories (latent space)")
		# self.ax_z0.set_xlabel('dim {}'.format(dim1))
		# self.ax_z0.set_ylabel('dim {}'.format(dim2))

		################################################
		# Show vector field
		self.ax_vector_field.cla()
		plot_vector_field(self.ax_vector_field, model.diffeq_solver.ode_func, model.latent_dim, device)
		self.ax_vector_field.set_title("Slice of vector field (latent space)", pad = 20)
		self.set_plot_lims(self.ax_vector_field, "vector_field")
		#self.ax_vector_field.set_ylim((-0.5, 1.5))

		################################################
		# Plot trajectories in the latent space

		# shape before [1, n_traj, n_tp, n_latent_dims]
		# Take only the first sample from approx posterior
		latent_traj = info["latent_traj"][0,:n_traj_to_show]
		# shape before permute: [1, n_tp, n_latent_dims]

		self.ax_latent_traj.cla()
		cmap = plt.colormaps['Accent']
		n_latent_dims = latent_traj.size(-1)

		custom_labels = {}
		for i in range(n_latent_dims):
			col = cmap(i)
			plot_trajectories(self.ax_latent_traj, latent_traj, time_steps_to_predict, 
				title="Latent trajectories z(t) (latent space)", dim_to_show = i, color = col, 
				marker = '', add_to_plot = True,
				linewidth = 3)
			custom_labels['dim ' + str(i)] = Line2D([0], [0], color=col)
		
		self.ax_latent_traj.set_ylabel("z")
		self.ax_latent_traj.set_title("Latent trajectories z(t) (latent space)", pad = 20)
		self.ax_latent_traj.legend(custom_labels.values(), custom_labels.keys(), loc = 'lower left')
		self.set_plot_lims(self.ax_latent_traj, "latent_traj")

		################################################

		self.fig.tight_layout()
		plt.draw()

		if save:
			dirname = "plots/" + str(experimentID) + "/"
			os.makedirs(dirname, exist_ok=True)
			self.fig.savefig(dirname + plot_name)
			plt.close()


In [33]:
weights = torch.load(f"/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/results_freq_noise/noise_both_noise_0.0/federated_training/FedAvg/100_rounds/rep_1-alpha_ 0.0-lr_0.001-batchsize_64_clipping_False_lrdecay_1.0_localepochs_10_decayonset_15_alpha_ 0.0/federated_outputs/model.pth", weights_only= True)
model_trained = model.load_state_dict(weights)

In [34]:
# make it not require gradients
# plot inline
%matplotlib inline
with torch.no_grad():
    v.draw_all_plots_one_dim(test_dict, model, plot_name = "testing", save = True, experimentID = 0.)

In [35]:
model.latent_dim

10